<a href="https://colab.research.google.com/github/paulynamagana/3D-Beacons/blob/main/Tutorials/Harnessing_3DBeaconsAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Accessing data though 3D-Beacons API</h1>

<img src="https://raw.githubusercontent.com/3D-Beacons/3D-Beacons/main/assets/3D-Beacons-logo.png" height="100" align="right">

## Introduction

Welcome to this Google Colab notebook, an essential companion to our paper on accessing data through the 3D-Beacons API.

This notebook serves as a practical resource for researchers, developers, and data enthusiasts who are interested in exploring the potential of the 3D-Beacons API and leveraging its capabilities. By following this guide, you will gain a deep understanding of how to interact with the 3D-Beacons API to access and analyze various types of data.
To supplement your learning, we have provided links to the full paper as well as documentation resources that will assist you in navigating the API effectively.

Documentation Link: [3D-Beacons API Documentation](https://www.ebi.ac.uk/pdbe/pdbe-kb/3dbeacons/api/#/default/get_uniprot_summary_uniprot_summary__qualifier__json_get)

<br>

**Further readings:**

**3D-Beacons: Decreasing the gap between protein sequences and structures through a federated network of protein structure data resources**

Mihaly Varadi, Sreenath Nair, Ian Sillitoe, Gerardo Tauriello, Stephen Anyango, Stefan Bienert, Clemente Borges, Mandar Deshpande, Tim Green, Demis Hassabis, Andras Hatos, Tamas Hegedus, Maarten L Hekkelman, Robbie Joosten, John Jumper, Agata Laydon, Dmitry Molodenskiy, Damiano Piovesan, Edoardo Salladini, Steven L. Salzberg, Markus J Sommer, Martin Steinegger, Erzsebet Suhajda, Dmitri Svergun, Luiggi Tenorio-Ku, Silvio Tosatto, Kathryn Tunyasuvunakool, Andrew Mark Waterhouse, Augustin Žídek, Torsten Schwede, Christine Orengo, Sameer Velankar
3 August 2022; BioRxiv https://doi.org/10.1101/2022.08.01.501973


  ## Instructions <a name="INSTRUCTIONS"></a>

* Quick Start <a name="Quick Start"></a>

In order to make the learning experience more accessible and interactive, we have incorporated widgets that allow you to provide inputs and customize certain aspects of the code. These widgets serve as interactive elements that enhance your ability to interact with the code and observe the impact of different inputs on the results. Throughout this tutorial, you will come across code chunks that generate these interactive widgets.

  * How to use Google Colab <a name="Google Colab"></a>
1. Before running the code, make sure you have filled the necessary information in the input form.
2. To run a code cell, click on the cell to select it. You will notice a play button (▶️) on the left side of the cell. Click on the play button or press Shift+Enter to run the code in the selected cell.
3. The code will start executing, and you will see the output, if any, displayed below the code cell.
4. Move to the next code cell and repeat steps 2 and 3 until you have executed all the desired code cells in sequence.
5. The currently running step is indicated by a circle with a stop sign next to it.
If you need to stop or interrupt the execution of a code cell, you can click on the stop button (■) located next to the play button.


*Remember to run the code cells in the correct order, as their execution might depend on variables or functions defined in previous cells. You can modify the code in a code cell and re-run it to see updated results.*

*Note: If the notebook runtime is restarted you will need to re-run the first 3 code chunks located in the Set Up section.*



## Contact us

Requests for clarifications or reporting bugs can be made via pdbekb_help@ebi.ac.uk.






---



In [15]:
#@title Set Up: Install libraries and mount Google Drive

#@markdown After running this code, you may be prompted to grant access to your Google Drive. This is necessary for Google Colab to download the files and save them to your Drive.
#@markdown <br>
#@markdown <br>
#@markdown Please follow the on-screen instructions to provide the necessary permissions, as it enables seamless integration between Colab and your Drive. Rest assured that your data and files are secure and will not be accessed without your explicit permission.

!pip install ijson gwpy &> /dev/null

print("Succesfully installed!")

#@title Mount Drive to download files
######## LIBRARIES
import ijson
import requests, sys, json
import ipywidgets as wgt
from urllib.request import urlopen
# Importing the necessary libraries for mounting Google Drive
import os
from google.colab import drive


######## FUNCTIONS
#defining functions for search and download
def Search3DBeacons(ID):
  WEBSITE_API = "https://www.ebi.ac.uk/pdbe/pdbe-kb/3dbeacons/api/v2/uniprot/summary/"

  r = ijson.parse(urlopen(f"{WEBSITE_API}{ID}.json"))
  structures = list(ijson.items(r, "structures.item", use_float=True))
  return structures


# Defining function for sequence search
def sequence_search(sequence):
    global job_id

    post_url = "https://www.ebi.ac.uk/pdbe/pdbe-kb/3dbeacons/api/v2/sequence/search"
    query_sequence = {"sequence": sequence}

    try:
        response = requests.post(post_url, json=query_sequence)
        response.raise_for_status()  # Raise an exception if it fails
        if response.status_code == 200:
            print("Your search was successful")
            job_id = response.json()["job_id"]
            retrieve_results(job_id)
        else:
            print(f"Request failed with status code {response.status_code}")
            exit()
    except requests.RequestException as e:
        print(f"Request failed with status code: {response.status_code}")
        print(f"Response text: {response.text}")
        exit()

def retrieve_results(job_id):
    get_url = "https://www.ebi.ac.uk/pdbe/pdbe-kb/3dbeacons/api/v2/sequence/result"
    try:
        # Use requests.get for better error handling
        response = requests.get(f"{get_url}?job_id={job_id}")
        response.raise_for_status()

        # Parse the JSON response with ijson
        items = ijson.items(response.content, 'item')
        for item in items:
            print(item)
    except requests.RequestException as e:
        print(f"An error occurred during the GET request: {e}")
        exit()


# Defining function for Ensembl search
def ensembl_search(ENSEMBL_ID):
    url = "https://www.ebi.ac.uk/pdbe/pdbe-kb/3dbeacons/api/v2/ensembl/summary/" #url

    try:
        # Send GET request to retrieve Ensembl summary
        response = urlopen(f"{url}{ENSEMBL_ID}.json")
        if response.status == 200:
            print("Ensembl search successful")
        else:
            print(f"Ensembl search failed with status code {response.status}")

        r = ijson.parse(response)
        ensembls = ijson.items(r, "uniprot_mappings.item", use_float=True)

        # Loop for better printing
        for ensembl in ensembls:
            print(ensembl)
    except Exception as e:
        print(f"An error occurred during the Ensembl search: {e}")
        exit()

######## API
#url_API
WEBSITE_API = "https://www.ebi.ac.uk/pdbe/pdbe-kb/3dbeacons/api/v2/uniprot/summary/"

######## GOOGLE DRIVE
# Mounting the Google Drive to access files and directories
drive.mount('/content/drive')
destination_path = '/content/drive/MyDrive/3D_Beacons_files'

# Check whether the specified path exists or not
isExist = os.path.exists(destination_path)
if not isExist:
   # Create a new directory because it does not exist
   os.makedirs(destination_path)
   print("The new directory is created!")

def download_file(model_url, destination_file_path):
  try:
    urllib.request.urlretrieve(model_url, destination_file_path)
    print(f'File downloaded successfully to: {destination_file_path}')
  except urllib.error.URLError as e:
    print(f'Error downloading file: {e}')  # Handle URL-related errors
  except Exception as e:  # Catch other general exceptions
    print(f'An unexpected error occurred: {e}')


Succesfully installed!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).




---



##1.&nbsp; BASIC SEARCH

This section implements API to search and filter predicted and experimentally determined structures available from the 3D-Beacons network.

Once you have found a model, you can download the specific

In [16]:
#@title 1.1.&nbsp;  Get all macromolecular structures for a single entry
#@markdown  On this sub-section you will be able to retrieve all available structures and metainformation in 3D-Beacons from a single Uniprot accession ID.
#Run this code to display the widget
Uniprot_ID = "P00520" #@param {type:"string"}

#Run this code to search the network using the input from the widget
structures = Search3DBeacons(Uniprot_ID)

#print available structures from different providers
for structure in structures:
    print(structure)

{'summary': {'model_identifier': '3kfa', 'model_category': 'EXPERIMENTALLY DETERMINED', 'model_url': 'https://www.ebi.ac.uk/pdbe/static/entry/3kfa_updated.cif', 'model_format': 'MMCIF', 'model_type': None, 'model_page_url': 'https://www.ebi.ac.uk/pdbe/entry/pdb/3kfa', 'provider': 'PDBe', 'number_of_conformers': None, 'ensemble_sample_url': None, 'ensemble_sample_format': None, 'created': '2009-10-27', 'sequence_identity': Decimal('100.0'), 'uniprot_start': 229, 'uniprot_end': 515, 'coverage': Decimal('0.256'), 'experimental_method': 'X-RAY DIFFRACTION', 'resolution': Decimal('1.22'), 'confidence_type': None, 'confidence_version': None, 'confidence_avg_local_score': None, 'oligomeric_state': None, 'preferred_assembly_id': None, 'entities': [{'entity_type': 'POLYMER', 'entity_poly_type': 'POLYPEPTIDE(L)', 'identifier': 'P00520', 'identifier_category': 'UNIPROT', 'description': 'Tyrosine-protein kinase ABL1', 'chain_ids': ['A', 'B']}, {'entity_type': 'NON-POLYMER', 'entity_poly_type': Non

### 1.2.&nbsp;  Performing one  filter

In [17]:
#@title 1.2.1.  Filter by Model type
#@markdown  Results can be filtered according to the model category in the 3D-Beacons Network. Models are classified as:

#@markdown * EXPERIMENTALLY DETERMINED
#@markdown * CONFORMATIONAL ENSEMBLE
#@markdown * TEMPLATE-BASED
#@markdown * AB-INITIO

#@markdown Code below allows you to filter the previous search using a model category.
#@markdown
model_type = "EXPERIMENTALLY_DETERMINED" #@param ["CONFORMATIONAL_ENSEMBLE", "EXPERIMENTALLY_DETERMINED", "TEMPLATE-BASED", "AB-INITIO"]
model_type_modified= model_type.replace("_", " ")

#Run this code to filter according to model category
for structure in structures:
    model_category = structure.get('summary', {}).get('model_category')
    if model_category == model_type_modified:
        print(structure)

{'summary': {'model_identifier': '3kfa', 'model_category': 'EXPERIMENTALLY DETERMINED', 'model_url': 'https://www.ebi.ac.uk/pdbe/static/entry/3kfa_updated.cif', 'model_format': 'MMCIF', 'model_type': None, 'model_page_url': 'https://www.ebi.ac.uk/pdbe/entry/pdb/3kfa', 'provider': 'PDBe', 'number_of_conformers': None, 'ensemble_sample_url': None, 'ensemble_sample_format': None, 'created': '2009-10-27', 'sequence_identity': Decimal('100.0'), 'uniprot_start': 229, 'uniprot_end': 515, 'coverage': Decimal('0.256'), 'experimental_method': 'X-RAY DIFFRACTION', 'resolution': Decimal('1.22'), 'confidence_type': None, 'confidence_version': None, 'confidence_avg_local_score': None, 'oligomeric_state': None, 'preferred_assembly_id': None, 'entities': [{'entity_type': 'POLYMER', 'entity_poly_type': 'POLYPEPTIDE(L)', 'identifier': 'P00520', 'identifier_category': 'UNIPROT', 'description': 'Tyrosine-protein kinase ABL1', 'chain_ids': ['A', 'B']}, {'entity_type': 'NON-POLYMER', 'entity_poly_type': Non

In [18]:
#@title 1.2.2.&nbsp; Filter by Provider

#@markdown The 3D-Beacons Network facilitates the aggregation of coordinate files and metadata for both experimental and theoretical protein models. It encompasses a wide range of state-of-the-art and specialized model providers, as well as data from the European Protein Data Bank (PDBe).

#@markdown Model providers:
#@markdown * PDBe
#@markdown * SWISS-MODEL
#@markdown * AlphaFold DB
#@markdown * Genome3D
#@markdown * SASBDB
#@markdown * AlphaFill
#@markdown * ModelArchive
#@markdown * Protein Ensemble Database (PED)

#@markdown For further information, visit the [Documentation](https://wwwdev.ebi.ac.uk/pdbe/pdbe-kb/3dbeacons/docs) to check all the current providers.


provider = "AlphaFill" #@param ["PDBe", "SWISS-MODEL", "AlphaFold_DB", "Genome3D", "SASBDB", "AlphaFill", "ModelArchive", "Protein_Ensemble_Database_(PED)"]
provider_modified= provider.replace("_", " ")

#Run this code to filter results according to given provider
for structure in structures:
    provider = structure.get('summary', {}).get('provider')

    if provider == provider_modified:
        print(structure)

{'summary': {'model_identifier': 'P00520', 'model_category': 'TEMPLATE-BASED', 'model_url': 'https://alphafill.eu/v1/aff/P00520', 'model_format': 'MMCIF', 'model_page_url': 'https://alphafill.eu/model?id=P00520', 'provider': 'AlphaFill', 'created': '2023-12-21', 'sequence_identity': Decimal('1.0'), 'uniprot_start': 1, 'uniprot_end': 1123, 'coverage': Decimal('1.0'), 'entities': [{'entity_type': 'POLYMER', 'entity_poly_type': 'POLYPEPTIDE(L)', 'description': 'Tyrosine-protein kinase ABL1', 'chain_ids': ['A']}, {'entity_type': 'NON-POLYMER', 'description': 'N-(2-CHLORO-6-METHYLPHENYL)-2-({6-[4-(2-HYDROXYETHYL)PIPERAZIN-1-YL]-2-METHYLPYRIMIDIN-4-YL}AMINO)-1,3-THIAZOLE-5-CARBOXAMIDE', 'chain_ids': ['B']}, {'entity_type': 'NON-POLYMER', 'description': '4-(4-METHYL-PIPERAZIN-1-YLMETHYL)-N-[4-METHYL-3-(4-PYRIDIN-3-YL-PYRIMIDIN-2-YLAMINO)-PHENYL]-BENZAMIDE', 'chain_ids': ['C', 'S', 'T']}, {'entity_type': 'NON-POLYMER', 'description': '3-(imidazo[1,2-b]pyridazin-3-ylethynyl)-4-methyl-N-{4-[(4-m

### 1.3.&nbsp;  Performing two filters

This section uses two sets of filters to display the models available in the network

In [19]:
#@title 1.3.1  Filter by Model type and Coverage

#markdown The code below will filter all the structures from a Uniprot ID and will save the model with the highest coverage to Google Drive.
import urllib.request

Uniprot_ID = "P04637"  # @param {type:"string"}
model_type = "CONFORMATIONAL_ENSEMBLE"  # @param ["CONFORMATIONAL_ENSEMBLE", "EXPERIMENTALLY_DETERMINED", "TEMPLATE-BASED", "AB-INITIO"]
model_type_modified = model_type.replace("_", " ")

#search and filter
response = urlopen(f"{WEBSITE_API}{Uniprot_ID}.json")
r = ijson.parse(response)
structures = list(ijson.items(r, "structures.item", use_float=True))
structures.sort(key=lambda x: x.get('summary', {}).get('coverage', 0), reverse=True)

highest_coverage_structure = None
for structure in structures:
    model_category = structure.get('summary', {}).get('model_category')
    if model_category == model_type_modified:
        highest_coverage_structure = structure
        break

# Check if a structure was found before accessing its properties
if highest_coverage_structure is not None:
    print( highest_coverage_structure)

else:
    print(f"No structure found with model type: {model_type_modified}")


{'summary': {'model_identifier': 'PED00438e001', 'model_category': 'CONFORMATIONAL ENSEMBLE', 'model_url': 'https://deposition.proteinensemble.org/api/v1/entries/PED00438/ensembles/e001/ensemble-sample/', 'model_format': 'MMCIF', 'model_page_url': 'https://proteinensemble.org/entries/PED00438', 'provider': 'PED', 'number_of_conformers': Decimal('101.0'), 'ensemble_sample_url': 'https://deposition.proteinensemble.org/api/v1/entries/PED00438/ensembles/e001/ensemble-sample/', 'ensemble_sample_format': 'MMCIF', 'created': '2020-10-01', 'sequence_identity': Decimal('0.9694656488549618'), 'uniprot_start': 1, 'uniprot_end': 393, 'coverage': Decimal('1.0'), 'entities': []}}


In [20]:
#@title 1.3.2&nbsp;  Filter by model type and retrieve results according to resolution.

#@markdown The code below will filter all the structures from a Uniprot ID and will save the models with a resolution higher than the specified on the widget.
# Run this code to display widgets
Uniprot_ID = "P04637"  # @param {type:"string"}
provider = "PDBe" #@param ["PDBe", "SWISS-MODEL", "AlphaFold_DB", "Genome3D", "SASBDB", "AlphaFill", "ModelArchive", "Protein_Ensemble_Database_(PED)"]
provider_modified= provider.replace("_", " ")
resolution_search = 1.3 # @param {type:"slider", min:0.1, max:5.0, step:0.1}

#Run this code to search and filter results according to the input provided
r = ijson.parse(urlopen(f"{WEBSITE_API}{Uniprot_ID}.json")) #build
structures = list(ijson.items(r, "structures.item", use_float=True))

#filter for experimental structures
high_resolution_structures = []
for structure in structures:
    provider = structure.get('summary', {}).get('provider')
    resolution = structure.get('summary', {}).get('resolution')

    if provider == provider_modified and resolution is not None and resolution < resolution_search:
        # Append the structure to the list without assigning the result back to the list
        high_resolution_structures.append(structure)

# Perform filter with the high-resolution structures
for structure in high_resolution_structures:
    print(structure)

{'summary': {'model_identifier': '3d06', 'model_category': 'EXPERIMENTALLY DETERMINED', 'model_url': 'https://www.ebi.ac.uk/pdbe/static/entry/3d06_updated.cif', 'model_format': 'MMCIF', 'model_type': None, 'model_page_url': 'https://www.ebi.ac.uk/pdbe/entry/pdb/3d06', 'provider': 'PDBe', 'number_of_conformers': None, 'ensemble_sample_url': None, 'ensemble_sample_format': None, 'created': '2008-05-01', 'sequence_identity': Decimal('99.0'), 'uniprot_start': 94, 'uniprot_end': 293, 'coverage': Decimal('0.509'), 'experimental_method': 'X-RAY DIFFRACTION', 'resolution': Decimal('1.2'), 'confidence_type': None, 'confidence_version': None, 'confidence_avg_local_score': None, 'oligomeric_state': None, 'preferred_assembly_id': None, 'entities': [{'entity_type': 'POLYMER', 'entity_poly_type': 'POLYPEPTIDE(L)', 'identifier': 'P04637', 'identifier_category': 'UNIPROT', 'description': 'Cellular tumor antigen p53', 'chain_ids': ['A']}, {'entity_type': 'NON-POLYMER', 'entity_poly_type': None, 'identi



---



##2.&nbsp; BASIC SEARCH

In [21]:
#@title Perform sequence search

#@markdown The 3D-Beacons Network has introduced sequence similarity search functionality which allows you to query the network using the amino acid sequence of a protein.

#@markdown The Sequence Similarity Search option available through the network uses the Basic Local Alignment Search Tool (BLAST, Altschul et al., 1990) to find regions of sequence similarity by aligning them with a query sequence. By evaluating the match between the network and query sequence, valuable insights into the structure, function, and evolutionary aspects can be obtained, thus facilitating targeted and systematic exploration of protein structures.

#@markdown The code presented below allows you to search the network by performing a sequence-based search via API.

sequence = "MNMLVINGTPRKHGRTRIAASYIAALYHTA"# @param {type:"string"}

#Run this code to perform sequence search, it will use the AA from the widget
sequence_search(sequence)

Your search was successful
{'accession': 'A0A809FN55', 'id': 'A0A809FN55_BACIU', 'description': 'NAD(P)H-dependent oxidoreductase OS=Bacillus subtilis subsp. subtilis OX=135461 GN=D9C10_03195 PE=3 SV=1', 'hit_length': 174, 'hit_hsps': [{'hsp_score': Decimal('150.0'), 'hsp_bit_score': Decimal('62.3882'), 'hsp_align_len': 29, 'hsp_identity': Decimal('100.0'), 'hsp_positive': Decimal('100.0'), 'hsp_qseq': 'MNMLVINGTPRKHGRTRIAASYIAALYHT', 'hsp_hseq': 'MNMLVINGTPRKHGRTRIAASYIAALYHT', 'hsp_mseq': 'MNMLVINGTPRKHGRTRIAASYIAALYHT', 'hsp_expect': Decimal('6.9E-11')}], 'summary': {'uniprot_entry': {'ac': 'A0A809FN55', 'id': 'A0A809FN55_BACIU', 'uniprot_checksum': 'DCD2A2EAA2677C1E', 'sequence_length': 174, 'segment_start': 1, 'segment_end': 174, 'description': None}, 'structures': [{'summary': {'model_identifier': 'AF-A0A809FN55-F1', 'model_category': 'AB-INITIO', 'model_url': 'https://alphafold.ebi.ac.uk/files/AF-A0A809FN55-F1-model_v4.cif', 'model_format': 'MMCIF', 'model_type': None, 'model_pa



---



##3.&nbsp; RETRIEVING ENSEMBL SUMMARY VIA 3D-BEACONS



In [22]:
#@title Retrieve summary using Ensembl Identifier
#@markdown The 3D-Beacons network provides the resource for retrieving Ensembl summaries to explore the diverse transcript variants associated with a given gene. ENSEMBL provides information about the different transcript variants for a given gene that map to an UniProt ID.

Ensembl_ID_search= "ENSG00000288864" # @param {type:"string"}

ensembl_search(Ensembl_ID_search)

Ensembl search successful
{'ensembl_transcript': {'transcript_id': '5507799', 'seqRegionStart': 32613860, 'seqRegionEnd': 32697077, 'alignment_difference': Decimal('0.0')}, 'uniprot_accession': {'uniprot_entry': {'ac': 'A0A8I5KS94', 'id': 'A0A8I5KS94_HUMAN', 'uniprot_checksum': 'EA44F0E7563755ED', 'sequence_length': 623, 'segment_start': 41, 'segment_end': 250, 'description': 'ARHGAP11A-SCG5 readthrough'}, 'structures': [{'summary': {'model_identifier': 'A0A8I5KS94_41-250:3eap.1.A', 'model_category': 'TEMPLATE-BASED', 'model_url': 'https://swissmodel.expasy.org/3d-beacons/uniprot/A0A8I5KS94.pdb?range=41-250&template=3eap.1.A&provider=swissmodel', 'model_format': 'PDB', 'model_type': 'ATOMIC', 'model_page_url': 'https://swissmodel.expasy.org/repository/uniprot/A0A8I5KS94?range=41-250&template=3eap.1.A', 'provider': 'SWISS-MODEL', 'created': '2024-02-15', 'sequence_identity': Decimal('1.0'), 'uniprot_start': 41, 'uniprot_end': 250, 'coverage': Decimal('0.337'), 'confidence_type': 'QMEAND



---



##4.&nbsp; DOWNLOAD SPECIFIC MODEL

The downloaded model can be found in the folder "3D_Beacons_files" in your Google Drive

In [29]:
#@title Download
#@markdown Once you found the model/ models you need, you can customise this code to download the MMCIF file to Google Drive

#@markdown If multiple models are required, separated them with a comma.
Uniprot_ID = "P04637"# @param {type:"string"}
model_download = "3d06, 4mzi"  # @param {type:"string"}
model_download_modified = ", ".join(model_download.strip().split(","))
#@markdown The model_download can be found in the key 'model_identifier' within every summary

structures = Search3DBeacons(Uniprot_ID)
### Run this code to download the models, it will use the input from the widget
for structure in structures:
  model = structure.get('summary', {}).get('model_identifier')
  if model in model_download_modified:
    model_url = structure.get("summary", {}).get("model_url")
    model_format = structure.get("summary", {}).get("model_format")
    file_extension = '.cif' if model_format == 'MMCIF' else '.pdb'
    destination_file_path = os.path.join(destination_path, model + file_extension)
    download_file(model_url, destination_file_path)


File downloaded successfully to: /content/drive/MyDrive/3D_Beacons_files/3d06.cif
File downloaded successfully to: /content/drive/MyDrive/3D_Beacons_files/4mzi.cif




---



# TROUBLESHOOTING

| Problem / Response code| Possible cause | Solution |
| :---------: |  :- | :- |
| 202 | **Accepted request.** <br> 1. The sequence search submission was correct, and the job has been assigned a job identifier. <br> 2. The sequence search job is currently running.| 1.Please wait until the sequence search run completes. It can take 5-10+ minutes.
| 400 | **Bad request** <br>  1. Malformed UniProt accession.  <br>  2. Possibly invalid sequence.  <br>  3. Job identifier not found.|1. Please check that the input UniProt accession is correct. <br>  2. Please check your input sequence and retry the submission. <br> 3. Please check if your job identifier is correct.
| 404| **Not found.** <br> No results found for the given request.| 1. There may be no results for a specific UniProt accession or protein sequence
|500| **Internal Server Error** | 1. This error might be due to scheduled maintenance or rarely technical issues. <br> 2. Please try again later. If the issue persists, please email pdbekb_help@ebi.ac.uk



# Bugs

If you encounter any bugs, please report the issue to pdbekb_help@ebi.ac.uk.
